# Análise Preditiva de Suicídios

## Descrição do dataset
Os dados foram coletados em forma de questionário através de redes sociais e emails. Alguns direcionados a grupos específicos de apoio (contra depressão). As questões abordadas foram baseadas nesta [pesquisa](https://www.kaggle.com/kingburrito666/the-demographic-rforeveralone-dataset). <br>

<b>Descrição das variáveis:</b><br>
* Q1: "Qual a sua idade? <br>
* Q2: "Qual gênero você mais se identifica?" <br>
* Q3: "Qual sua orientação  sexual?" <br>
* Q4: "Qual o seu nível de renda anual?" <br>
* Q5: "Qual sua etnicidade?" <br>
* Q6: "Como você descreve sua massa corporal?" <br>
* Q7: "Você é virgem?" <br>
* Q8: "Você pagaria para ter uma relação sexual?" <br>
* Q9: "Quantos amigos você tem na vida real?" <br>
* Q10: "Você possui ansiedade/fobia social?" <br>
* Q11: "Você possui algum tipo de diagnóstico de transtorno mental?" <br>
* Q12: "Que tipo de ajuda você quer? <br>
* Q13: "Você já tentou se suicidar?" <br>
* Q14: "Qual seu status empregatício? Atualmente você é..." <br>
* Q15: "Qual seu cargo atual?" <br>
* Q16: "Qual seu nível de educação?" <br>
* Q17: "O que você fez para se sentir melhor consigo mesmo?" <br>
* Q18: "Com quem você mora?" <br>
* Q19: "Seus pais são divorciados?" <br>


In [263]:
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 100)

In [264]:
survey_data = pd.read_csv('survey_data.csv')

In [265]:
df = survey_data.iloc[:, 1:20]
df.columns = ['Q{}'.format(x+1) for x in range(0, 19)]
df.isnull().sum()

Q1     0  
Q2     0  
Q3     0  
Q4     6  
Q5     0  
Q6     0  
Q7     0  
Q8     0  
Q9     0  
Q10    0  
Q11    84 
Q12    0  
Q13    0  
Q14    0  
Q15    111
Q16    0  
Q17    0  
Q18    0  
Q19    0  
dtype: int64

## 1. Pré-processamento

In [266]:
def check_values_column(series):
    return pd.DataFrame(series.str.strip().unique()).sort_values(by=0)

In [267]:
def replace_values(df, column_name, values_search, value_replace):
    for value in values_search:
        contains_value = df[column_name].str.lower().str.contains(value)
        df[column_name] = np.where(contains_value == True, value_replace, df[column_name])

### 1.1 Variável Q4

In [268]:
df[df['Q4'].isna()].shape

(6, 19)

In [269]:
df['Q4'].value_counts()

R$1,00 - R$9,999         101
R$10,000 - R$19,999      42 
R$20,000 - R$29,999      29 
R$50,000 - R$74,999      19 
R$200,000 ou mais        12 
R$30,000 - R$39,999      11 
R$75,000 - R$99,999      7  
R$40,000 - R$49,999      7  
R$125,000 - R$149,999    4  
R$100,000 - R$124,999    3  
R$175,000 - R$199,999    2  
R$150,000 - R$174,999    1  
Name: Q4, dtype: int64

#### 1.1.1 Substituindo valores pela moda

Baseado na análise, os registros são referente a estudantes e/ou desempregados

In [270]:
mode = df['Q4'].value_counts().index[0]

In [271]:
df['Q4'].fillna(mode, inplace=True)

### 1.2 Variável Q11
Esta variável envolve multipla seleção e digitação em campo livre.<br>
Os passos para tratamento serão:<br>
* Atribuir valor caso não tenha sido informado;
* Sobrescrever valores redundantes (digitados);
* Converter multipla seleção em colunas separadas;
* Converter as classes separadas para binário;
* Classificar valores digitados manualmente em categorias existentes e/ou criando novas (se necessário);

In [272]:
df[df['Q11'].isna()].shape

(84, 19)

In [273]:
df['Q11'].fillna('Sem resposta', inplace=True)

In [274]:
check_values_column(df['Q11'])

,0
21,Ansiedade de leve
12,"Apenas períodos de ansiedade + depressão , mas nao transtornos que me acompanharam durante a vida"
28,Nenhum
38,Nenhum (que eu saiba).
33,Nenhum diagnóstico
23,Não
39,Não possuo diagnóstico
30,Não tenho como responder pois evito procurar ajuda
45,"Não tenho nada diagnosticado, mas me sinto constantemente sobrecarregado e extremamente estressado. Pelo que leio, isso se chama Sindrome de Burn Out."
35,Não tenho transtorno mental


#### 1.2.1 Agrupando/sobrescrevendo valores redundantes

In [275]:
nenhum = 'Nenhum'

df.loc[df.Q11.str.lower().str.contains('nenhum'), 'Q11'] = nenhum
df.loc[df.Q11.str.lower().str.contains('não'), 'Q11'] = nenhum
df.loc[df.Q11.str.lower().str.contains('sem diagnóstico médico'), 'Q11'] = nenhum

outros = 'Outros'
df.loc[df.Q11.str.lower().str.contains('ansiedade de leve'), 'Q11'] = outros
df.loc[df.Q11.str.lower().str.contains('apenas períodos de ansiedade'), 'Q11'] = outros
df.loc[df.Q11.str.lower().str.contains('problemas psicológicos após lesão corporal'), 'Q11'] = outros
df.loc[df.Q11.str.lower().str.contains('tdah'), 'Q11'] = outros

df.Q11 = df.Q11.str.strip().str.replace('toc', 'TOC', regex=True)

In [276]:
check_values_column(df['Q11'])

,0
21,Nenhum
11,Outros
1,Sem resposta
0,"Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...)"
29,"Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...);Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)"
19,"Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...);Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)"
15,"Transtornos Alimentares (Anorexia, Bulimia, Transtorno de Ruminação, etc...)"
34,"Transtornos Alimentares (Anorexia, Bulimia, Transtorno de Ruminação, etc...);Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...);Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)"
18,"Transtornos Alimentares (Anorexia, Bulimia, Transtorno de Ruminação, etc...);Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)"
26,"Transtornos Alimentares (Anorexia, Bulimia, Transtorno de Ruminação, etc...);Transtornos de Ansiedade (Fobia Específica, Transtorno de Ansiedade Generalizada, Transtorno de Pânico, etc...)"


#### 1.2.2 Transformando variáveis categóricas em binário
Criando colunas binárias separadas para cada tipo de transtorno.

In [277]:
df_Q11 = pd.get_dummies(df['Q11'].str.strip().str.split(';', expand=True))

df['Q11_T1'] = df_Q11[['0_Transtornos Alimentares  (Anorexia, Bulimia, Transtorno de Ruminação, etc...)']].max(axis=1)

df['Q11_T2'] = df_Q11[['0_Transtornos de Ansiedade (Fobia Específica, Transtorno de Ansiedade Generalizada, Transtorno de Pânico, etc...)',
                       '1_Transtornos de Ansiedade (Fobia Específica, Transtorno de Ansiedade Generalizada, Transtorno de Pânico, etc...)']].max(axis=1)

df['Q11_T3'] = df_Q11[['0_Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...)',
                       '1_Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...)',
                       '2_Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...)']].max(axis=1)

df['Q11_T4'] = df_Q11[['0_Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)',
                       '1_Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)',
                       '2_Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)',
                       '3_Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)']].max(axis=1)

df['Q11_T5'] = df_Q11[['0_Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)',
                       '1_Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)',
                       '2_Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)',
                       '3_Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)',
                       '4_Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)']].max(axis=1)

df['Q11_T6'] = df_Q11[['0_Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)',
                       '1_Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)',
                       '2_Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)',
                       '3_Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)',
                       '4_Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)',
                       '5_Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)']].max(axis=1)

df['Q11_T7'] = df_Q11[['1_TOC', '2_TOC', '5_TOC']].max(axis=1)

In [278]:
df['Q11_T8'] = df_Q11[['0_Outros']].max(axis=1)
df['Q11_T9'] = df_Q11[['0_Sem resposta']].max(axis=1)

In [279]:
df = df.drop(columns='Q11')

In [280]:
df.columns

Index(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q12',
       'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q11_T1', 'Q11_T2',
       'Q11_T3', 'Q11_T4', 'Q11_T5', 'Q11_T6', 'Q11_T7', 'Q11_T8', 'Q11_T9'],
      dtype='object')

### 1.2 Variável Q15

In [281]:
df['Q15'].str.strip().value_counts()

Estudante                                               20
Estagiário                                              4 
Professor                                               3 
Estagiária                                              3 
Analista de sistema                                     2 
Vendedora                                               2 
Gerente                                                 2 
Bolsista                                                2 
professora                                              2 
Desenvolvedor de Sistemas                               2 
Dev                                                     2 
Auxiliar de cartório                                    2 
Recepcionista                                           2 
analista de sistemas                                    2 
Desenvolvedor de Software                               2 
Estágio                                                 2 
Cientista de Dados                                      

#### 1.3.1 Agrupando valores

In [295]:
desenvolvimento = ['desenvolvedor', 'dev', 'analista', 'software', 'inteligência artificial']
ciencia_de_dados = ['dados', 'estatístico']
estudante = ['estudante', 'caloura', 'bolsista', 'escola', 'terceiro ano']
professor = ['professor', 'ministro aula casualmente']
estagiario = ['estagiári', 'estágio']
vendas = ['vendedor', 'vendas', 'atendente', 'call center', 'marketing', 'operador de caixa']
recepcionista = ['recepcionista', 'recepçao']
autonomo = ['dono', 'autônoma', 'empreendedor', 'empresária']
lideranca = ['gerente', 'diretor', 'líder']
redator = ['redator']
pesquisa = ['pesquisador']
psicologia = ['psic']
coordernadoria = ['coordenad']
adm = ['adm']

replace_values(df, 'Q15', desenvolvimento, 'Desenvolvimento de Software')
replace_values(df, 'Q15', estudante, 'Estudante')
replace_values(df, 'Q15', estagiario, 'Estágio')
replace_values(df, 'Q15', professor, 'Educação')
replace_values(df, 'Q15', vendas, 'Vendas / Telemarketing')
replace_values(df, 'Q15', recepcionista, 'Recepcionista')
replace_values(df, 'Q15', autonomo, 'Empresário(a) / Empreendedor(a) / Autônomo(a)')
replace_values(df, 'Q15', ciencia_de_dados, 'Ciência de Dados')
replace_values(df, 'Q15', lideranca, 'Diretoria / Gerência / Liderança')
replace_values(df, 'Q15', redator, 'Redator(a)')
replace_values(df, 'Q15', psicologia, 'Psicologia')
replace_values(df, 'Q15', pesquisa, 'Pesquisador(a)')
replace_values(df, 'Q15', coordernadoria, 'Coordenador(a)')
replace_values(df, 'Q15', coordernadoria, 'Administrativo')

In [296]:
df['Q15'].str.strip().value_counts()

Estudante                                                      31
Desenvolvimento de Software                                    26
Estágio                                                        11
Vendas / Telemarketing                                         11
Educação                                                       10
Diretoria / Gerência / Liderança                               5 
Recepcionista                                                  4 
Empresário(a) / Empreendedor(a) / Autônomo(a)                  4 
Ciência de Dados                                               3 
Administrativo                                                 3 
Redator(a)                                                     2 
Auxiliar de cartório                                           2 
Fotógrafo                                                      1 
Cat Sitter, Auxiliar de Veterinária e Curso de banho e tosa    1 
Freelancer                                                     1 
Auxiliar F